<a href="https://colab.research.google.com/github/SWazniewicz/UMwF-1-/blob/main/Zadanie_3(2)_UMwF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stwórz model generujący sygnały kupna i sprzedaży na rynku
Decyzje ma generować jeden z algorytmów uczenia maszynowego, po optymalizacji hiperparametrów
Zrób backtest wybranego modelu, użyj biblioteki przystosowanej do backtestów
Pamiętaj o wnioskach i wizualizacji wyników
4 pkt z oceny będą zależne od wyników inwestycji
Inwestujesz w wylosowaną spółkę przez określony okres, ale można wykorzystać także inne dane niż historyczne (np. obliczone wskaźniki)

Strategia inwestycyjna (decyzje wejścia i wyjścia) dla spółki  Microsoft (MSFT), test w okresie od 01.01.2024 - 06.05.2024


In [ ]:
!pip install yfinance backtesting scikit-learn ta --quiet

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import ta

from backtesting import Backtest, Strategy
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from scipy.stats import randint, uniform

In [ ]:
symbol = "MSFT"
start_train = "2020-01-01"
end_all = "2024-05-06"
test_start = "2024-01-01"
test_end = "2024-05-06"

data = yf.download(symbol, start=start_train, end=end_all)
data.columns = data.columns.get_level_values(0)
data.head()

In [ ]:
print(data.columns)
print(data.head())

In [ ]:
df = data.copy()

df["return_1d"] = df["Close"].pct_change()
df["return_3d"] = df["Close"].pct_change(3)
df["return_5d"] = df["Close"].pct_change(5)
df["return_10d"] = df["Close"].pct_change(10)

df["sma_10"] = df["Close"].rolling(10).mean()
df["sma_20"] = df["Close"].rolling(20).mean()
df["sma_50"] = df["Close"].rolling(50).mean()
df["ema_20"] = df["Close"].ewm(span=20).mean()

df["atr_14"] = ta.volatility.average_true_range(df["High"], df["Low"], df["Close"], window=14)
df["bb_high"] = ta.volatility.BollingerBands(df["Close"], window=20).bollinger_hband()
df["bb_low"] = ta.volatility.BollingerBands(df["Close"], window=20).bollinger_lband()
df["bb_width"] = (df["bb_high"] - df["bb_low"]) / df["Close"]

df["rsi_14"] = ta.momentum.rsi(df["Close"], window=14)
df["stoch_k"] = ta.momentum.stoch(df["High"], df["Low"], df["Close"], window=14, smooth_window=3)
df["adx"] = ta.trend.adx(df["High"], df["Low"], df["Close"], window=14)

macd = ta.trend.MACD(df["Close"])
df["macd"] = macd.macd()
df["macd_signal"] = macd.macd_signal()
df["macd_diff"] = macd.macd_diff()

df["vol_sma_10"] = df["Volume"].rolling(10).mean()
df["vol_sma_20"] = df["Volume"].rolling(20).mean()
df["vol_ratio"] = df["Volume"] / df["vol_sma_20"]

df.dropna(inplace=True)
df.head()

In [ ]:
df["future_return_3d"] = df["Close"].pct_change(3).shift(-3)

RET_TH = 0.002
df["target"] = (df["future_return_3d"] > RET_TH).astype(int)
df.dropna(inplace=True)

In [ ]:
test_start = "2024-01-01"
test_end = "2024-05-06"

feature_cols = [
    "return_1d", "return_3d", "return_5d", "return_10d",
    "sma_10", "sma_20", "sma_50", "ema_20",
    "atr_14", "bb_width",
    "rsi_14", "stoch_k", "adx",
    "macd", "macd_signal", "macd_diff",
    "vol_ratio"
]

X = df[feature_cols]
y = df["target"]

train_mask = df.index < test_start
test_mask = (df.index >= test_start) & (df.index <= test_end)

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", HistGradientBoostingClassifier(random_state=42))
])

param_distributions = {
    "clf__max_depth": randint(2, 8),
    "clf__learning_rate": uniform(0.01, 0.2),
    "clf__max_iter": randint(100, 500),
    "clf__min_samples_leaf": randint(10, 60),
    "clf__l2_regularization": uniform(0.0, 1.0)
}

search = RandomizedSearchCV(
    pipe,
    param_distributions=param_distributions,
    n_iter=40,
    scoring="roc_auc",
    cv=tscv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X_train, y_train)
best_model = search.best_estimator_

print("Najlepsze parametry:", search.best_params_)
print("Test ROC-AUC:", search.best_score_)

df["proba"] = best_model.predict_proba(X)[:, 1]

print(classification_report(y_test, (df.loc[test_mask, "proba"] > 0.5).astype(int)))

In [ ]:
from backtesting import Backtest


bt_df_full = df.copy()
bt_df_full["signal"] = 0

bt_data_test = bt_df_full.loc[test_start:test_end, ["Open","High","Low","Close","Volume","proba"]].copy()

In [ ]:
class MLStrategyTP_SL(Strategy):
    proba_entry = 0.55
    sl_atr = 1.8
    tp_atr = 3.0

    def init(self):
        pass

    def next(self):
        proba = self.data.proba[-1]

        atr = df.loc[self.data.index[-1], "atr_14"]

        if not self.position:
            if proba > self.proba_entry:
                sl = self.data.Close[-1] - self.sl_atr * atr
                tp = self.data.Close[-1] + self.tp_atr * atr
                self.buy(sl=sl, tp=tp)

        else:
            if proba < 0.45:
                self.position.close()


def run_bt_for_threshold(th):
    MLStrategyTP_SL.proba_entry = th
    bt = Backtest(
        bt_data_test,
        MLStrategyTP_SL,
        cash=10000,
        commission=0.001,
        exclusive_orders=True
    )
    stats = bt.run()
    return stats

thresholds = np.arange(0.50, 0.71, 0.02)
results = []

for th in thresholds:
    s = run_bt_for_threshold(th)
    results.append((th, s["Return [%]"], s["Sharpe Ratio"], s["# Trades"], s["Max. Drawdown [%]"]))

res_df = pd.DataFrame(results, columns=["threshold","Return%","Sharpe","#Trades","MaxDD%"])
print(res_df.sort_values("Return%", ascending=False).head(10))

best_th = res_df.sort_values("Return%", ascending=False).iloc[0]["threshold"]
print("Najlepszy próg:", best_th)

In [ ]:
MLStrategyTP_SL.proba_entry = float(best_th)

bt_final = Backtest(
    bt_data_test,
    MLStrategyTP_SL,
    cash=10000,
    commission=0.001,
    exclusive_orders=True
)

stats_final = bt_final.run()
print(stats_final)
bt_final.plot()

Utworzony model przewiduje, czy 3-dniowa stopa zwrotu akcji MSFT przekroczy 0,2%. Model trenowano na danych 2020–2023 z wykorzystaniem wskaźników technicznych (m.in. RSI, MACD, ATR, Bollinger Bands, ADX, średnie kroczące) oraz optymalizacji hiperparametrów metodą RandomizedSearchCV z walidacją TimeSeriesSplit.
Uzyskany wynik ROC-AUC ≈ 0,56 wskazuje na niewielką, ale dodatnią przewagę predykcyjną nad losowym zgadywaniem. Dokładność ok. 52% i umiarkowane wartości precision/recall pokazują, że przewidywanie kierunku krótkoterminowych ruchów cen jest trudne i obarczone szumem rynkowym. W strategii handlowej wykorzystano prawdopodobieństwo predykcji (proba) oraz zarządzanie ryzykiem SL/TP w oparciu o ATR. Dodatkowo zoptymalizowano próg wejścia – najlepszy był 0,56. Zwrot strategii wyniósł 8,66% i był lepszy od strategii Buy&Hold (5,17%).